In [1]:
import regex as re
import os
from collections import Counter

In [2]:
input_path = "data/testcase.txt" 
special_tokens = ["<|endoftext|>"]
vocab_size = 260

text = ""
with open(input_path, 'r') as f:
     text = f.read()

vocab = {}
for i in range(256):
    vocab[i] = bytes([i])

special_token_bytes = [st.encode('utf-8') for st in special_tokens]
for special_token in special_token_bytes:
    vocab[len(vocab)] = special_token

words = {}
for word in text.split():
    word_bytes = tuple(bytes([b]) for b in word.encode('utf-8'))
    # print(word_bytes)
    words[word_bytes] = words.get(word_bytes, 0) + 1

num_merges = vocab_size - 256
merges = []

for _ in range(num_merges):
    pair_freq = Counter()
    for word, freq in words.items():
        for i in range(len(word) - 1):
            pair = (word[i], word[i+1])
            pair_freq[pair] += freq

    # if not pair_freq: break

    best_pair = max(pair_freq, key= lambda p: (pair_freq[p], p))
    merges.append(best_pair)

    new_token = best_pair[0] + best_pair[1]
    vocab[len(vocab)] = new_token

    new_words = {}
    for word, freq in words.items():
        new_word = []
        i = 0
        while i < len(word):
            if (i<len(word)-1 and word[i]==best_pair[0] and word[i+1]==best_pair[1]):
                new_word.append(new_token)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        # print(new_word)
        new_words[tuple(new_word)] = freq
    words = new_words





In [ ]:
from bpe_trainer_naive import run_train_bpe as run_train_bpe_naive
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()

vocab, merge = run_train_bpe_naive(
    input_path="/Users/ryanli/Documents/repos/assignment1-basics/cs336_basics/data/tinystories_sample_5M.txt",
    vocab_size=10000,
    special_tokens=["<|endoftext|>"]
)

profiler.disable()

pstats.Stats(profiler).sort_stats('cumulative').print_stats()